In [32]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [46]:
def create_input():
    X = []
    y = []
    df = pd.read_csv("classification_data/explore_img/explore_data.csv")
    shuffled_df = df.sample(len(df))

    print(shuffled_df.head)
    s = 0
    for entry in shuffled_df.values:
        s += 1
        # print(s)
        entry_img = cv2.imread(f"classification_data/explore_img/{entry[0]}")

        prepared_y = np.delete(entry,0) * 3
        print(prepared_y)
        if prepared_y[0] >= 960:
            prepared_y[0] = prepared_y[0] - 960
        else:
            prepared_y[0] = -prepared_y[0]
            
        if prepared_y[1] >= 540:
            prepared_y[1] = prepared_y[1] - 540
        else:
            prepared_y[1] = -prepared_y[1]
        X.append(entry_img)
        print(prepared_y, 'normalized')
        y.append(prepared_y)
        
    
    X = np.asarray(X)
    y = np.asarray(y).astype(np.float32)
    return X, y


In [34]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer=Adam())

    return model

model = compile_model()

In [35]:
model.compile(optimizer='adam',
              loss='mean_squared_error')

In [47]:
X, y = create_input()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)


<bound method NDFrame.head of             filename    x    y
77    explore_77.jpg  100  268
21    explore_21.jpg   40   65
10    explore_10.jpg  274   26
15    explore_15.jpg  182   38
153  explore_153.jpg  223  272
..               ...  ...  ...
5      explore_5.jpg  212  286
61    explore_61.jpg  248  124
144  explore_144.jpg  455   60
87    explore_87.jpg  360   28
72    explore_72.jpg  354  338

[160 rows x 3 columns]>
[300 804]
[-300 264] normalized
[120 195]
[-120 -195] normalized
[822 78]
[-822 -78] normalized
[546 114]
[-546 -114] normalized
[669 816]
[-669 276] normalized
[1374 216]
[414 -216] normalized
[1638 234]
[678 -234] normalized
[480 786]
[-480 246] normalized
[288 483]
[-288 -483] normalized
[1491 264]
[531 -264] normalized
[1611 219]
[651 -219] normalized
[984 69]
[24 -69] normalized
[951 990]
[-951 450] normalized
[1233 954]
[273 414] normalized
[1242 978]
[282 438] normalized
[372 285]
[-372 -285] normalized
[1401 183]
[441 -183] normalized
[525 114]
[-525 -114] no

In [48]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))


Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 953ms/step - loss: 453050.5000 - val_loss: 195647.4062
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 874ms/step - loss: 165213.6094 - val_loss: 177615.7344
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 894ms/step - loss: 133957.4844 - val_loss: 150834.0312
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 891ms/step - loss: 111931.4531 - val_loss: 147836.6562
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 939ms/step - loss: 95340.2344 - val_loss: 140455.5938
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 889ms/step - loss: 62191.3203 - val_loss: 145608.9219
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 854ms/step - loss: 40861.3828 - val_loss: 147992.5000
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 948ms/step - loss: 35584.3125 - val_loss: 153237.4688
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 894ms/step - loss: 24880.5605 - val_loss: 135535.1719
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 885ms/step - loss: 19461.7227 - val_loss: 140572.5312
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 823ms/step - loss: 15685.

In [49]:
model.save('mouse_explore_fhd_normalized.h5')

In [50]:
model_rap = tf.keras.models.load_model('mouse_explore_fhd_normalized.h5')


for n in range(20):
    x_real, y_real = int(y_val[n][0]), int(y_val[n][1])
    print(x_real, y_real)
    img = np.asarray(X_val[n])

    tbp = np.expand_dims(img, axis=0)
    img = cv2.resize(img,(1920,1080))
    predictions = model_rap.predict(tbp)
    x, y = int(predictions[0][0]), int(predictions[0][1])
    if x < 0:
        x = -x 
    else:
        x = x + 960
    
    if y < 0:
        y = -y
    else:
        y = y + 540
    
    print(x, y, "\n",x_real, y_real)
    
    cv2.circle(img, (x,y), 10, (0,0,255), -1)
    cv2.circle(img, (x_real, y_real), 10, (255,255,255), -1)
    
    cv2.imshow('ss', img)
    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

318 -285
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1183 62 
 318 -285
879 225
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1607 739 
 879 225
-522 -105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
555 185 
 -522 -105
-807 510
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
165 661 
 -807 510
-384 117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
706 663 
 -384 117
444 -234
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1094 63 
 444 -234
303 -141
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1257 169 
 303 -141
-738 441
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
351 819 
 -738 441
576 111
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
16 548 
 576 111
282 -117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
50 146 
 282 -117
-645 -216
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
742 297 
 -645 -216
627 -303
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1197 12 
 627 -303
-372 -285
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
245 146 
 -372 -285
747 99
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1373 681 
 747 99
-771 447
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
36 654 
 -771 447
837 -495
1/

In [41]:
model_rap = tf.keras.models.load_model('mouse_explore_fhd2.h5')
model_rap.compile(optimizer='adam',
              loss='mean_squared_error')


model_rap.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))
model_rap.save('mouse_explore_fhd2.h5')

Epoch 1/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 2466981.0000 - val_loss: 1351475.7500
Epoch 2/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 943ms/step - loss: 1013773.5625 - val_loss: 221797.5000
Epoch 3/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 878ms/step - loss: 167532.5938 - val_loss: 207102.3125
Epoch 4/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 871ms/step - loss: 123114.7422 - val_loss: 225627.2500
Epoch 5/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 910ms/step - loss: 121374.1250 - val_loss: 190306.0312
Epoch 6/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - loss: 122928.6406 - val_loss: 199820.6719
Epoch 7/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 903ms/step - loss: 108252.2812 - val_loss: 195921.4375
Epoch 8/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 872ms/step - loss: 103571.0703 - val_loss: 183029.7500
Epoch 9/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 869ms/step - loss: 84680.7656 - val_loss: 193496.6875
Epoch 10/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 894ms/step - loss: 81365.6562 - val_loss: 184376.2188
Epoch 11/30
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 924ms/step - loss: 67383